Соединение и аутентификация

При удачном подключении к почте увидим сообщение [b'Authentication successful']

In [45]:
import imaplib
import email
from email.header import decode_header
import base64
from bs4 import BeautifulSoup
import re

def read(sender_of_interest=None):
    # Очищаем файл перед записью
    with open('filtered_emails.txt', 'r+') as f:
        f.truncate(0) 

    # Подключаемся к серверу
    imap = imaplib.IMAP4_SSL("imap.gmail.com", 993)
    imap.login('remboplas@gmail.com', 'trok ekct masw xlwz')
    imap.select('INBOX')

    # Поиск всех писем
    if sender_of_interest:
        status, response = imap.uid('search', None, 'ALL', 'FROM {0}'.format(sender_of_interest))
    else:
        status, response = imap.uid('search', None, 'ALL')

    if status == 'OK':
        unread_msg_nums = response[0].split()
        print(f"Found {len(unread_msg_nums)} emails")  # Отладочная информация
    else:
        unread_msg_nums = []
        print("No emails found")

    # Инвертируем список ID, чтобы получить последние письма первыми
    unread_msg_nums = unread_msg_nums[::-1]

    data_list = []
    count = 0  # Счётчик писем

    for e_id in unread_msg_nums:
        if count >= 200:  # Ограничение на 200 последних писем
            break

        e_id = e_id.decode('utf-8')
        _, response = imap.uid('fetch', e_id, '(RFC822)')

        # Используем message_from_bytes для правильного парсинга писем
        raw_email = response[0][1]
        email_message = email.message_from_bytes(raw_email)

        # Проверка, multipart или нет, чтобы обработать текстовые и HTML части
        if email_message.is_multipart():
            for part in email_message.walk():
                content_type = part.get_content_type()
                if content_type == 'text/plain':  # Извлекаем только текстовую часть
                    body = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                    data_list.append(body)
                    print(f"Added text/plain part from email {e_id}")  # Отладочная информация
                    count += 1
                    break  # Если нашли нужную часть, выходим
        else:
            # Если не multipart, просто получаем тело письма
            body = email_message.get_payload(decode=True).decode('utf-8', errors='ignore')
            data_list.append(body)
            
            count += 1

    # Запись в файл
    with open('filtered_emails.txt', 'w', encoding='utf-8') as f:
        for elem in data_list:
            # Удаляем блоки стилей <style>...</style>
            elem = re.sub(r"<style.*?>.*?</style>", "", elem, flags=re.S)
            elem = re.sub(r"&nbsp;", "", elem, flags=re.S)
            # Удаляем все HTML-теги
            elem = re.sub(r"<[^>]+>", "", elem, flags=re.S)
            
            if 'Высылаю обратную связь' in elem:
                print(f"Added non-multipart email {e_id}")  # Отладочная информация

                # Разбиваем на строки и записываем строки, если они не пустые
                for line in elem.splitlines():
                    if line.strip():  # Проверяем, не пустая ли строка
                        f.write(line + '\n')  # Записываем в файл

                # Добавляем разделитель для каждого письма
                f.write('\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')

    # Завершаем работу с сервером
    imap.logout()

# Вызов функции
read()


Found 1518 emails
Added text/plain part from email 4801
Added text/plain part from email 4800
Added text/plain part from email 4799
Added text/plain part from email 4798
Added text/plain part from email 4797
Added text/plain part from email 4796
Added text/plain part from email 4795
Added text/plain part from email 4794
Added text/plain part from email 4793
Added text/plain part from email 4792
Added text/plain part from email 4791
Added text/plain part from email 4790
Added text/plain part from email 4789
Added text/plain part from email 4788
Added text/plain part from email 4784
Added text/plain part from email 4783
Added text/plain part from email 4782
Added text/plain part from email 4781
Added text/plain part from email 4780
Added text/plain part from email 4779
Added text/plain part from email 4778
Added text/plain part from email 4777
Added text/plain part from email 4776
Added text/plain part from email 4775
Added text/plain part from email 4773
Added text/plain part from email

In [50]:
import re

import nltk

# Попытка загрузки стоп-слов
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Загрузка стоп-слов для русского языка (предварительно нужно выполнить nltk.download('stopwords'))
stop_words = set(stopwords.words('russian'))

def clean_text(text):
    # Удаление HTML-тегов
    text = re.sub(r'<.*?>', '', text)
    
    # Удаление всех символов, кроме букв и пробелов
    text = re.sub(r'[^а-яА-ЯёЁa-zA-Z ]', '', text)
    
    # Приведение текста к нижнему регистру
    text = text.lower()
    
    # Токенизация и удаление стоп-слов
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    
    # Объединяем очищенные токены обратно в строку
    clean_text = ' '.join(filtered_text)
    
    return clean_text

# Открытие файла filtered_emails.txt для чтения
with open('filtered_emails.txt', 'r', encoding='utf-8') as f:
    raw_data = f.read()

# Разделение текста по строкам с плюсиками
emails = raw_data.split('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\\n')

# Очистка каждого письма
cleaned_emails = [clean_text(email) for email in emails]

# Запись очищенного текста в новый файл
with open('cleaned_emails.txt', 'w', encoding='utf-8') as f:
    for email in cleaned_emails:
        f.write(email + '\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\\n')

print("Очистка завершена. Очищенный текст записан в 'cleaned_emails.txt'.")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sanya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\sanya/nltk_data'
    - 'c:\\Users\\sanya\\anaconda3\\nltk_data'
    - 'c:\\Users\\sanya\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\sanya\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\sanya\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************
